<a href="https://colab.research.google.com/github/nmwiley808/csci167/blob/main/Notebooks/Chap12/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [4]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  query = np.dot(omega_q, x) + beta_q
  key = np.dot(omega_k, x) + beta_k
  value = np.dot(omega_v, x) + beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [5]:
def softmax(items_in):
  # Exponentiate each element
  exp_items = np.exp(items_in - np.max(items_in))  # Subtract max for numerical stability
  # Normalize by dividing by the sum of exponentials
  items_out = exp_items / np.sum(exp_items)

  return items_out ;

Now compute the self attention values:

In [6]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # TODO -- compute the appropriate dot product
    # Replace this line
    dot_product = 1

    # Store dot product
    all_km_qn.append(dot_product)

  # Compute dot product
  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  print("Attentions for output ", n)
  print(attention)

  x_prime = np.zeros((D,1))
  for i, value in enumerate(all_values):
      # Weighted sum of values according to attention weights
      x_prime += attention[i] * value

  all_x_prime.append(x_prime)



# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[0.33333333 0.33333333 0.33333333]
Attentions for output  1
[0.33333333 0.33333333 0.33333333]
Attentions for output  2
[0.33333333 0.33333333 0.33333333]
x_prime_0_calculated: [[-0.03953915  1.07543197 -1.26480747 -0.42754445]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[-0.03953915  1.07543197 -1.26480747 -0.42754445]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[-0.03953915  1.07543197 -1.26480747 -0.42754445]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [7]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [8]:
 # Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # 1. Compute queries, keys, and values
  Q = np.array([sum(omega_q[i][j] * X[j] for j in range(len(X))) + beta_q[i] for i in range(len(omega_q))])
  K = np.array([sum(omega_k[i][j] * X[j] for j in range(len(X))) + beta_k[i] for i in range(len(omega_k))])
  V = np.array([sum(omega_v[i][j] * X[j] for j in range(len(X))) + beta_v[i] for i in range(len(omega_v))])


  # Step 2: Compute dot products (Q * K^T)
  dot_product = np.dot(Q, K.T)  # (N, N), Queries and Keys

  # Step 3: Apply softmax across columns to get attention weights
  attention_weights = softmax_cols(dot_product)  # (N, N) attention matrix

  # Step 4: Weight values by attention weights
  X_prime = np.dot(attention_weights, V)  # (N, D)

  return X_prime

In [9]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 1.28555857e+00 -8.81778530e-02 -3.02993313e-02]
 [-1.26594881e+01 -4.23120166e-01  7.89494364e+00]
 [ 2.26825811e+00 -1.55581640e-01 -5.34626829e-02]
 [-1.63113830e-02 -2.08751617e-03  1.03910096e-02]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [10]:

# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # 1. Compute queries, keys, and values
  Q = np.matmul(omega_q, X) + beta_q
  K = np.matmul(omega_k, X) + beta_k
  V = np.matmul(omega_v, X) + beta_v

  # Step 2: Compute dot products
  dot_product = np.matmul(K.T, Q)  # (N, N), Queries and Keys
  D_k = K.shape[0]
   # 3. Scale the dot products
  scaled_dot_product = dot_product / np.sqrt(D_k)

  # 4. Apply softmax to calculate attentions
  attention_weights = softmax_cols(scaled_dot_product)
  # 5. Weight values by attentions
  X_prime = np.matmul(V, attention_weights)


  return X_prime.T

In [11]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.97411966 -0.23738409 -0.72333202 -0.34413007]
 [ 1.59622051 -0.09516106  3.70194096  2.01339538]
 [ 1.32638014  0.13062402  3.02371664  1.6902419 ]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.

# Answer
Yes, the self-attention mechanism is permutation-equivariant, meaning that permuting the input tokens matrix with tokens as columns and P is a permutation matrix, then the self-attention output satisfies
Sa[XP] = Sa[X]P. This holds becasue the the key, query, and value matrices K,Q,V are all linear transformations of X; therefore, permuting X simply permutes the columns of each. The attention scors K^TQ transform as p^T(K^TQ)P, and since the softmax is applied column-wise, it preserves this permytation structure. As a result, the attention operation treats all positions identically, and permuting the inputs only reorders the outputs. However, this covariance is broken when positional encodings or attention masks are introduced, since these explicitly depend on token order and thus destroy permutation symmetry.
